# 모델 이해하기

## 표기 방법 1

- 합성곱 층 : 합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)
- 풀링 층 : 맥스풀링(nn.MaxPoold2d)

## 표기 방법 2 (<- 우리가 사용할 방법)

- 합성곱 층 : 합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)

```py
# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax
```

# 모델 구현

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# 배치 크기 * 채널 * 높이 * 너비 크기 텐서 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서 크기 : {}'.format(inputs.shape))

텐서 크기 : torch.Size([1, 1, 28, 28])


## 합성곱층과 풀링 선언

In [ ]:
# 첫번째 합성곱 층 구현
conv1 = nn.Conv2d(1, 32, 3, padding=1) # 입력:1채널짜리, 출력:32채널짜리, 커널사이즈 3, 패딩 1
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
# 두번째 합성곱 층 구현
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
# 맥스풀링 구현
pool = nn.MaxPool2d(2) # 인자가 하나일 경우 커널 사이즈와 스트라이드 둘 다 해당값으로 지정
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 구현체 연결해 모델 만들기

In [ ]:
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [ ]:
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [ ]:
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [ ]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [ ]:
# .size(n) 은 n번째 차원의 크기를 보여준다.
out.size(0)

1

In [ ]:
# .view()를 사용하여 텐서 펼치기
# 첫번째 차원(배치 차원) 제외 나머지 모두 펼치기
out = out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

In [ ]:
fc = nn.Linear(3136, 10)
out = fc(out)
out.shape

torch.Size([1, 10])

# CNN으로 MNIST 분류

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 랜덤 시드 고정
torch.manual_seed(777)
# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [16]:
# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로
                          train=True, # 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [18]:
# 데이터로더로 배치 크기 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [19]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [20]:
# CNN 모델 정의
model = CNN().to(device)

In [21]:
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch)) # batch size = 100, 600*100=60000

총 배치의 수 : 600


In [23]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())